In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
#prepare the dataset
data = datasets.load_breast_cancer()
X,y = data.data, data.target
n_samples, n_features = X.shape
#print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1) #for reshaping y in column vector
y_test = y_test.view(y_test.shape[0],1)

In [10]:
#design model
# y= wx + b , sigmoid function at the end
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
model = LogisticRegression(n_features)
#loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#training loop
num_epoch = 100
for epoch in range(num_epoch):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    #backward pass
    loss.backward()
    #update weights
    optimizer.step()
    #zero gradient
    optimizer.zero_grad()
    if (epoch+1)%10 ==0:
        print(f'epoch:{epoch+1}, loss={loss.item():.4f}')
#evaluation
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = y_pred.round()
    acc = y_pred_class.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy:{acc:.4f}')
    
    

epoch:10, loss=0.4763
epoch:20, loss=0.4220
epoch:30, loss=0.3825
epoch:40, loss=0.3524
epoch:50, loss=0.3285
epoch:60, loss=0.3089
epoch:70, loss=0.2926
epoch:80, loss=0.2787
epoch:90, loss=0.2668
epoch:100, loss=0.2563
accuracy:0.9649


In [13]:
loss = nn.CrossEntropyLoss()
Y = torch.tensor([0])
Y_pred_good = torch.tensor([[2.0,1.0,0.1]])
Y_pred_bad = torch.tensor([[0.5,2.0,0.3]])
l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)
print(l1.item())
print(l2.item())

0.4170299470424652
1.840616226196289
